#Archivo para limpieza de los datos generados por el SAMTM

El esquema de la base de datos necesario es el siguiente:
  - id_evento
  - operador [sin desarrollar]
  - turno [sin desarrollar]
  - maquina
  - causa
  - inicio
  - fin
  - duración [seg]

## Librerías

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## 1. Cargar datos
Para poder cargar los datos deben de estar en el mismo entorno de ejecución

El esquema de los datos se presenta de la siguiente forma:

  P2:

    - fecha, Formato DD/MM/AAAA
    - hora, Formato HH:MM:SS
    - Maquina 1,
    - Maquina 2, estados de las máquina [1,2,3]
    - Maquina 3,
    - Maquina 4:10, no utilizados

  Los estados de las máquinas corresponden:

    1- Producción
    2- Paro
    3- Preparación

  P3:

    - fecha, Formato DD/MM/AAAA
    - hora, Formato HH:MM:SS
    - Maquina 1,
    - Maquina 2, en este campo se captura el id de causa de paro
    - Maquina 3,
    - Maquina 4:10, no utilizados


El archivo "catalogoCausas.csv" muestra la base de causas con su id correspondiente.

In [40]:
# Cargar datos, se creará un df por cada .csv importado -----

dataP2 = pd.read_csv("P2.csv")
dataP3 = pd.read_csv("P3.csv")
causas = pd.read_csv("catalogoCausas.csv")


## 2. Limpiar datos
Consta de tres etapas:
  1. Eliminar columnas innecesarias.
  2. Detectar cambios de estado y eliminar eventos sin cambios.
  3. Calcular duración del evento.

In [3]:
# Crear df de datos limpios con su estructura según el esquema [turno y operador aún no se agregan]

dfClean = {"idEvento":[],
           "maquina":[],
           "causa":[],
           "inicio":[],
           "fin":[],
           "duracion":[]}
dfClean = pd.DataFrame(dfClean)
#dfClean


In [41]:
# 3.1 Unificar columnas de fecha y hora, formatear a datetime -----
# Tabla P2

dataP2['marcaTemporal'] = pd.to_datetime(dataP2['fecha'] + " " + dataP2['hora'])
dataP2 = dataP2[['marcaTemporal', 'Maquina 1', 'Maquina 2', 'Maquina 3']]

# Tabla P3

dataP3['marcaTemporal'] = pd.to_datetime(dataP3['fecha'] + " " + dataP3['hora'])
dataP3 = dataP3[['marcaTemporal', 'Maquina 1', 'Maquina 2', 'Maquina 3']]



In [43]:
dataP2[["Maquina 2", "Maquina 2", "Maquina 3"]] = dataP3[["Maquina 2", "Maquina 2", "Maquina 3"]]
dataP2

ValueError: Columns must be same length as key

In [30]:
# 3.2 Detectar cambios de estado -----

for i in ["Maquina 1", "Maquina 2", "Maquina 3"]:
  dataP2[f"cambio_{i[-1]}"] = dataP2[i].ne(dataP2[i].shift())

# Mantener solo los eventos donde hubo al menos un cambio -----

dataP2 = dataP2[dataP2[['cambio_1','cambio_2','cambio_3']].any(axis=1)].copy()

# 3.3 Calcular duración hacia el siguiente evento -----

dataP2["siguienteMarca"] = dataP2["marcaTemporal"].shift(-1)
dataP2["duracion"] = (dataP2["siguienteMarca"] - dataP2["marcaTemporal"])/pd.Timedelta(seconds=1)


In [ ]:



# 3.2 Detectar cambios de estado -----

for m in ['Maquina 1','Maquina 2','Maquina 3']:
    dataP2[f'cambio_{m[-1]}'] = dataP2[m].ne(dataP2[m].shift())

# Mantener solo los eventos donde hubo al menos un cambio -----

dfClean= dataP2[dataP2[['cambio_1','cambio_2','cambio_3']].any(axis=1)].copy()

# 3.3 Calcular duración hacia el siguiente evento -----

dfClean['siguienteMarca'] = dfClean['marcaTemporal'].shift(-1)
dfClean['duracion'] = dfClean['siguienteMarca'] - dfClean['marcaTemporal']

In [ ]:
dfClean

,marcaTemporal,Maquina 1,Maquina 2,Maquina 3,cambio_1,cambio_2,cambio_3,siguienteMarca,duracion
0,2025-12-12 07:23:10,2,2,2,True,True,True,2025-12-12 07:24:11,0 days 00:01:01
2,2025-12-12 07:24:11,2,3,2,False,True,False,2025-12-12 07:27:24,0 days 00:03:13
8,2025-12-12 07:27:24,2,3,3,False,False,True,2025-12-12 07:36:08,0 days 00:08:44
24,2025-12-12 07:36:08,3,3,3,True,False,False,2025-12-12 07:39:57,0 days 00:03:49
31,2025-12-12 07:39:57,2,1,1,True,True,True,2025-12-12 07:43:05,0 days 00:03:08
37,2025-12-12 07:43:05,3,1,1,True,False,False,2025-12-12 07:44:09,0 days 00:01:04
39,2025-12-12 07:44:09,1,1,1,True,False,False,2025-12-12 07:44:39,0 days 00:00:30
40,2025-12-12 07:44:39,2,1,1,True,False,False,2025-12-12 07:47:53,0 days 00:03:14
46,2025-12-12 07:47:53,3,1,1,True,False,False,2025-12-12 07:49:25,0 days 00:01:32
49,2025-12-12 07:49:25,1,1,1,True,False,False,2025-12-12 07:52:37,0 days 00:03:12


## 4. Convertir a .csv

In [ ]:
dfClean["marcaTemporal"] = dfClean["marcaTemporal"].dt.strftime("%Y%m%d%H%M%S")

dfClean["siguienteMarca"] = dfClean["siguienteMarca"].dt.strftime("%Y%m%d%H%M%S")

#dfClean["duracion"] = dfClean["duracion"].dt.strftime("%Y%m%d%H%M%S")

In [ ]:
dfClean.to_csv("datos.csv", index=False)